# Augment distribution properties - WFD

In this notebook we computee what are the properties of the test set, and hence which properties the augmented training set should have. We perform this analysis for WFD test set.

#### Index<a name="index"></a>
1. [Import Packages](#imports)
2. [Load Dataset](#loadData)
3. [Compute Dataset Distributions](#distr)
    1. [Spectroscopic Redshift Distributions](#distrSpecz)
    2. [Target Number Observations](#distrNumberObs)
    3. [Observations Uncertainty](#distrUnc)
        1. [u](#uDistr) [g](#gDistr) [r](#rDistr) [i](#iDistr) [z](#zDistr) [y](#yDistr)
    4. [SNR/ Detections](#distrDect)
    5. [Number and lenght of Detections](#distrDetect)

## 1. Import Packages<a name="imports"></a>

In [ ]:
!pip install ../snmachine/

In [ ]:
import collections
import os
import pickle
import sys
import time

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
from sklearn.mixture import GaussianMixture
from snmachine import snaugment, sndata
from utils.plasticc_pipeline import get_directories, load_dataset

In [ ]:
%config Completer.use_jedi = False  # enable autocomplete

#### Aestetic settings

In [ ]:
%matplotlib inline

sns.set(font_scale=1.3, style="ticks")

## 2. Load Dataset<a name="loadData"></a>

First, **write** the path to the folder that contains the dataset we want to augment, `folder_path`.

In [ ]:
# os_name = 'baseline_v2_0_paper'
# os_name = 'noroll_v2_0_paper'
os_name = 'presto_v2_0_paper'

folder_path = f'/folder/path/'
analyses_dir = f'/analyses/path'

Then, **write** in `data_file_name` the name of the file where your dataset is saved.

In this notebook we use the dataset saved in [2_preprocess_data]().

In [ ]:
is_only_roll = 0
is_updated = 1

In [ ]:
extra_name_to_save = 'wfd'

file_id = '000'
#file_id = '002' # until 009

train_data_file_name = f'train_ddf_{extra_name_to_save}_{file_id}_gapless50.pckl'
test_data_file_name = f'test_{extra_name_to_save}_{file_id}_gapless50.pckl'
if is_only_roll:
    train_data_file_name = f'train_ddf_{extra_name_to_save}_{file_id}_roll_gapless50.pckl'
    test_data_file_name = f'test_{extra_name_to_save}_{file_id}_roll_gapless50.pckl'
if is_updated:
    train_data_file_name = train_data_file_name[:-5] + '_updated.pckl'
    test_data_file_name = test_data_file_name[:-5] + '_updated.pckl'
test_data_file_name

Load the dataset.

In [ ]:
data_path = os.path.join(folder_path, train_data_file_name)
train_data = load_dataset(data_path)

[Go back to top.](#index)

## 3. Compute Dataset Distributions<a name="distr"></a>

### 3.1. Spectroscopic Redshift Distributions<a name="distrSpecz"></a>

First we check that all the events have spectroscopic redshift. Any event without this value should have been removed in previous notebooks.

Then, we model the redhsift distributions using Gaussian Mixture Models.

In [ ]:
diverg_color = sns.color_palette("Set2", 6, desat=1)
sn_type_color = {42: diverg_color[1], 62: diverg_color[0], 90: diverg_color[2], 
                 52: diverg_color[3], 67: diverg_color[4], 95: diverg_color[5]}
sn_type_name = {42: 'SN II', 62: 'SN Ibc', 90: 'SN Ia', 
                52: 'SN Iax', 67: 'SN 91bg', 95: 'SLSN'}
unique_types = [90, 42, 62, 52, 67, 95]

unique_types = [90, 42, 62]

In [ ]:
bins = np.linspace(0, 3.2, 50)
g = sns.distplot(a=train_metadata['hostgal_photoz'], kde=True, norm_hist=True,
                 label='train data')
g = sns.distplot(a=test_metadata['hostgal_photoz'], kde=True, norm_hist=True,
                 label='test data')
plt.ylabel('Density')
plt.xlabel('Photometric z')
plt.legend()

In [ ]:
plt.figure(figsize=(10, 4))
for sn_type in unique_types:
    is_sn_type = (train_metadata['target'] == sn_type)
    sn_type_metadata = train_metadata[is_sn_type]
    bins = np.linspace(0, 3.2, 50)
    g = sns.distplot(a=sn_type_metadata['hostgal_photoz'], kde=True, hist=False,
                     label=sn_type_name[sn_type], color=sn_type_color[sn_type])
    
    
    is_sn_type = (test_metadata['target'] == sn_type)
    sn_type_metadata = test_metadata[is_sn_type]
    bins = np.linspace(0, 3.2, 50)
    g = sns.distplot(a=sn_type_metadata['hostgal_photoz'], kde=True, hist=False, 
                     color=sn_type_color[sn_type], kde_kws={'ls':'--'})
plt.plot([4,5], [0,1], 'k-', label='Train set')
plt.plot([4,5], [0,1], 'k--', label='Test set')
plt.xlabel('Photometric redshift')
plt.ylabel('Density')
#plt.title('Diff. between subsets')
plt.xlim(-.3, 3.5)
plt.ylim(0, 2.5)
plt.legend()

[Go back to top.](#index)

### 3.2. Target Number Observations <a name="distrNumberObs"></a>

First we compute the number of observations in each light curve.

In [ ]:
def compute_number_obs(dataset):
    obj_names = dataset.object_names
    number_obs = np.zeros(len(obj_names))
    for i in np.arange(len(obj_names)):
        obj = obj_names[i]
        obj_data = dataset.data[obj].to_pandas()
        number_obs[i] = np.shape(obj_data)[0]
    return number_obs

In [ ]:
train_metadata['number_obs'] = compute_number_obs(train_data)
test_metadata['number_obs'] = compute_number_obs(test_data)

In [ ]:
print(np.min(train_metadata['number_obs']), np.min(test_metadata['number_obs']))
print(np.max(train_metadata['number_obs']), np.max(test_metadata['number_obs']))
print(np.mean(train_metadata['number_obs']), np.mean(test_metadata['number_obs']))

Then, we model the number of observations using Gaussian Mixture Models. 

In [ ]:
data_to_fit = np.array(test_metadata['number_obs']).reshape(len(test_objs), 1)
gmm = GaussianMixture(n_components=2, random_state=0).fit(data_to_fit)
print(gmm.weights_)
print(gmm.means_)
print(gmm.covariances_)

In [ ]:
bins = np.linspace(0, 222, 75)
g = sns.distplot(a=train_metadata['number_obs'], kde=True, norm_hist=True,
                 label='train', bins=bins)
g = sns.distplot(a=test_metadata['number_obs'], kde=True, norm_hist=True,
                 label='test', bins=bins)

# Plot the estimated distribution
number_points = 100
x = np.linspace(0, 222, number_points).reshape(number_points, 1)
logprob = gmm.score_samples(x)
pdf = np.exp(logprob)
plt.plot(x+2, pdf, '-k', label='GMM fit')
plt.legend()

plt.ylabel('Density')
plt.xlabel('Total number of observations')

[Go back to top.](#index)

### 3.3. Observations Uncertainty <a name="distrUnc"></a>

First we compute the uncertainty in each passband for each light curve.

In [ ]:
def make_big_pb_unc_table(dataset, pb, subset=None):
    if subset is None:
        obj_names = dataset.object_names
    else:
        obj_names = dataset.object_names[subset]
    metadata = dataset.metadata
    unc_pb = []
    obs_target = []
    for obj in obj_names:
        obj_data = dataset.data[obj].to_pandas()
        is_pb = obj_data['filter'] == pb
        obj_data_pb = obj_data[is_pb]
        unc_pb.append(obj_data_pb['flux_error'])
        obs_target.append(len(obj_data_pb) * [metadata.loc[obj, 'target']])
    unc_pb = pd.concat(unc_pb, ignore_index=True)
    obs_target = pd.DataFrame([inner for outer in obs_target for inner in outer])
    return unc_pb, obs_target

In [ ]:
unc_train = []
unc_test = []
obs_target_train = []
obs_target_test = []
for pb in train_data.filter_set:
    unc_pb, obs_target_pb = make_big_pb_unc_table(train_data, pb)
    unc_train.append(unc_pb)
    obs_target_train.append(obs_target_pb)
    
    unc_pb, obs_target_pb = make_big_pb_unc_table(test_data, pb)
    unc_test.append(unc_pb)
    obs_target_test.append(obs_target_pb)
u_unc_train, g_unc_train, r_unc_train, i_unc_train, z_unc_train, y_unc_train = unc_train
u_unc_test, g_unc_test, r_unc_test, i_unc_test, z_unc_test, y_unc_test = unc_test
(u_obs_target_train, g_obs_target_train, r_obs_target_train, 
 i_obs_target_train, z_obs_target_train, y_obs_target_train) = obs_target_train
(u_obs_target_test, g_obs_target_test, r_obs_target_test, 
 i_obs_target_test, z_obs_target_test, y_obs_target_test) = obs_target_test

#### 3.3.1. u<a name="uDistr"></a>

In [ ]:
data_to_fit = np.array(np.log(u_unc_test)).reshape(len(u_unc_test), 1)
gmm = GaussianMixture(n_components=2, random_state=0).fit(data_to_fit)
print(gmm.weights_)
print(gmm.means_)
print(gmm.covariances_)

In [ ]:
bins = np.linspace(np.log(.4), np.log(30), 50)
g = sns.distplot(a=np.log(u_unc_train), kde=True, norm_hist=True,
                 label='train', bins=bins)
g = sns.distplot(a=np.log(u_unc_test), kde=True, norm_hist=True,
                 label='test', bins=bins)

# Plot the estimated distribution
number_points = 500
x = np.linspace(-1, 5, number_points).reshape(number_points, 1)
logprob = gmm.score_samples(x)
pdf = np.exp(logprob)
plt.plot(x, pdf, '-k')
plt.legend()
plt.title(f'All SN')
plt.ylabel('Density')
plt.xlim(0, 5)

In [ ]:
gauss_choice = np.random.choice(2, size=5000, p=gmm.weights_)
gmm_means = gmm.means_
gmm_covs = gmm.covariances_

gmm_sample = np.zeros(len(gauss_choice))
for i in np.arange(2):
    is_i = gauss_choice == i
    gmm_sample[is_i] = np.random.normal(gmm_means[i], np.sqrt(gmm_covs[i][0]), size=np.sum(is_i))

In [ ]:
bins = np.linspace(np.log(.4), np.log(30), 50)
g = sns.distplot(a=np.log(u_unc_train), kde=True, norm_hist=True,
                 label='train', bins=bins)
g = sns.distplot(a=np.log(u_unc_test), kde=True, norm_hist=True,
                 label='test', bins=bins)

s = np.exp(gmm_sample)
g = sns.distplot(a=np.log(s), kde=True, norm_hist=True,
                 label='log', bins=bins)
plt.xlim(np.log(1), np.log(30))
plt.legend()
plt.title(f'All SN')
plt.ylabel('Distribution')

#### 3.3.2. g<a name="gDistr"></a>

We model the uncertainty of observations using Gaussian Mixture Models. 

In [ ]:
print(np.min(np.log(g_unc_test)), np.max(np.log(g_unc_test)))
print(np.min(np.log(g_unc_train)), np.max(np.log(g_unc_train)))

In [ ]:
data_to_fit = np.array(np.log(g_unc_test)).reshape(len(g_unc_test), 1)
gmm = GaussianMixture(n_components=2, random_state=0).fit(data_to_fit)
print(gmm.weights_)
print(gmm.means_)
print(gmm.covariances_)

In [ ]:
bins = np.linspace(np.log(.4), np.log(30), 50)
g = sns.distplot(a=np.log(g_unc_train), kde=True, norm_hist=True,
                 label='train', bins=bins)
g = sns.distplot(a=np.log(g_unc_test), kde=True, norm_hist=True,
                 label='test', bins=bins)

# Plot the estimated distribution
number_points = 1000
x = np.linspace(-1, 5, number_points).reshape(number_points, 1)
logprob = gmm.score_samples(x)
pdf = np.exp(logprob)
plt.plot(x, pdf, '-k')
plt.legend()
plt.title(f'All SN')
plt.ylabel('Density')
plt.xlim(-1, 5)

In [ ]:
gauss_choice = np.random.choice(2, size=5000, p=gmm.weights_)
gmm_means = gmm.means_
gmm_covs = gmm.covariances_

gmm_sample = np.zeros(len(gauss_choice))
for i in np.arange(2):
    is_i = gauss_choice == i
    gmm_sample[is_i] = np.random.normal(gmm_means[i], np.sqrt(gmm_covs[i][0]), size=np.sum(is_i))

In [ ]:
bins = np.linspace(np.log(.4), np.log(30), 50)
g = sns.distplot(a=np.log(g_unc_train), kde=True, norm_hist=True,
                 label='train', bins=bins)
g = sns.distplot(a=np.log(g_unc_test), kde=True, norm_hist=True,
                 label='test', bins=bins)

s = np.exp(gmm_sample)
g = sns.distplot(a=np.log(s), kde=True, norm_hist=True,
                 label='log', bins=bins)
plt.xlim(-1, 3)
plt.legend()
plt.title(f'All SN')
plt.ylabel('Density')
plt.xlim(-1, 5)

#### 3.3.3. r<a name="rDistr"></a>

We model the uncertainty of observations using Gaussian Mixture Models. 

In [ ]:
print(np.min(np.log(r_unc_test)), np.max(np.log(r_unc_test)))
print(np.min(np.log(r_unc_train)), np.max(np.log(r_unc_train)))

In [ ]:
data_to_fit = np.array(np.log(r_unc_test)).reshape(len(r_unc_test), 1)
gmm = GaussianMixture(n_components=2, random_state=0).fit(data_to_fit)
print(gmm.weights_)
print(gmm.means_)
print(gmm.covariances_) 

In [ ]:
bins = np.linspace(np.log(.4), np.log(50), 50)
g = sns.distplot(a=np.log(r_unc_train), kde=True, norm_hist=True,
                 label='train', bins=bins)
g = sns.distplot(a=np.log(r_unc_test), kde=True, norm_hist=True,
                 label='test', bins=bins)

# Plot the estimated distribution
number_points = 1000
x = np.linspace(-1, 5, number_points).reshape(number_points, 1)
logprob = gmm.score_samples(x)
pdf = np.exp(logprob)
plt.plot(x, pdf, '-k')
plt.legend()
plt.title(f'All SN')
plt.ylabel('Density')
plt.xlim(-1, 5)

#### 3.3.4. i<a name="iDistr"></a>

We model the uncertainty of observations using Gaussian Mixture Models. 

In [ ]:
print(np.min(np.log(i_unc_test)), np.max(np.log(i_unc_test)))
print(np.min(np.log(i_unc_train)), np.max(np.log(i_unc_train)))

In [ ]:
data_to_fit = np.array(np.log(i_unc_test)).reshape(len(i_unc_test), 1)
gmm = GaussianMixture(n_components=2, random_state=0).fit(data_to_fit)
print(gmm.weights_)
print(gmm.means_)
print(gmm.covariances_)

In [ ]:
bins = np.linspace(np.log(.4), np.log(30), 50)
g = sns.distplot(a=np.log(i_unc_train), kde=True, norm_hist=True,
                 label='train', bins=bins)
g = sns.distplot(a=np.log(i_unc_test), kde=True, norm_hist=True,
                 label='test', bins=bins)

# Plot the estimated distribution
number_points = 500
x = np.linspace(-1, 5, number_points).reshape(number_points, 1)
logprob = gmm.score_samples(x)
pdf = np.exp(logprob)
plt.plot(x, pdf, '-k')
plt.legend()
plt.title(f'All SN')
plt.ylabel('Density')
plt.xlim(-0.5, 5)

#### 3.3.5. z<a name="zDistr"></a>

We model the uncertainty of observations using Gaussian Mixture Models. 

In [ ]:
print(np.min(np.log(z_unc_test)), np.max(np.log(z_unc_test)))
print(np.min(np.log(z_unc_train)), np.max(np.log(z_unc_train)))

In [ ]:
data_to_fit = np.array(np.log(z_unc_test)).reshape(len(z_unc_test), 1)
gmm = GaussianMixture(n_components=1, random_state=0).fit(data_to_fit)
print(gmm.weights_)
print(gmm.means_)
print(gmm.covariances_)

In [ ]:
bins = np.linspace(np.log(.4), np.log(500), 50)
g = sns.distplot(a=np.log(z_unc_train), kde=True, norm_hist=True,
                 label='train', bins=bins)
g = sns.distplot(a=np.log(z_unc_test), kde=True, norm_hist=True,
                 label='test', bins=bins)

# Plot the estimated distribution
number_points = 500
x = np.linspace(-1, 5, number_points).reshape(number_points, 1)
logprob = gmm.score_samples(x)
pdf = np.exp(logprob)
plt.plot(x, pdf, '-k')
plt.legend()
plt.title(f'All SN')
plt.ylabel('Density')
plt.xlim(0, 5)

#### 3.3.6. y<a name="yDistr"></a>

We model the uncertainty of observations using Gaussian Mixture Models. 

In [ ]:
print(np.min(np.log(y_unc_test)), np.max(np.log(y_unc_test)))
print(np.min(np.log(y_unc_train)), np.max(np.log(y_unc_train)))

In [ ]:
data_to_fit = np.array(np.log(y_unc_test)).reshape(len(y_unc_test), 1)
gmm = GaussianMixture(n_components=1, random_state=0).fit(data_to_fit)
print(gmm.weights_)
print(gmm.means_)
print(gmm.covariances_)

In [ ]:
bins = np.linspace(np.log(.4), np.log(500), 50)
g = sns.distplot(a=np.log(y_unc_train), kde=True, norm_hist=True,
                 label='train', bins=bins)
g = sns.distplot(a=np.log(y_unc_test), kde=True, norm_hist=True,
                 label='test', bins=bins)

# Plot the estimated distribution
number_points = 500
x = np.linspace(-1, 5, number_points).reshape(number_points, 1)
logprob = gmm.score_samples(x)
pdf = np.exp(logprob)
plt.plot(x, pdf, '-k')
plt.legend()
plt.title(f'All SN')
plt.ylabel('Density')
plt.xlim(0, 5)

[Go back to top.](#index)

### 3.4. SNR/ Detections <a name="distrDect"></a>

First we compute the SNR for each observation.

In [ ]:
def compute_obs_snr(dataset):
    objs = dataset.object_names
    obs_flux = []
    obs_flux_error = []
    for i, obj in enumerate(objs):
        obj_data = dataset.data[obj].to_pandas()
        obs_flux.append(obj_data['flux'])
        obs_flux_error.append(obj_data['flux_error'])
    obs_flux = np.concatenate(obs_flux)
    obs_flux_error = np.concatenate(obs_flux_error)
    obs_snr = obs_flux_error/obs_flux
    return obs_snr

In [ ]:
train_obs_snr = compute_obs_snr(train_data)
test_obs_snr = compute_obs_snr(test_data)

In [ ]:
log_obs_snr = np.log(train_obs_snr)
sns.distplot(log_obs_snr, label='train')
log_obs_snr = np.log(test_obs_snr)
sns.distplot(log_obs_snr, label='test')
plt.legend()

[Go back to top.](#index)

### 3.5. Number and lenght of detections <a name="distrDetect"></a>

In [ ]:
def compute_number_len_detect(dataset):
    obj_names = dataset.object_names
    number_detect = np.zeros(len(obj_names))
    len_detect = np.zeros(len(obj_names))
    for i in np.arange(len(obj_names)):
        obj = obj_names[i]
        obj_data = dataset.data[obj]
        is_detect = obj_data['detected'] == True
        number_detect[i] = np.sum(is_detect)
        
        obj_mjd_detect = obj_data['mjd'][is_detect]
        len_detect[i] = obj_mjd_detect[-1] - obj_mjd_detect[0]
    return number_detect, len_detect

In [ ]:
number_detect, len_detect = compute_number_len_detect(train_data)
train_metadata['number_detect'] = number_detect
train_metadata['len_detect'] = len_detect

In [ ]:
number_detect, len_detect = compute_number_len_detect(test_data)
test_metadata['number_detect'] = number_detect
test_metadata['len_detect'] = len_detect

In [ ]:
print(np.min(train_metadata['number_detect']), np.min(test_metadata['number_detect']))
print(np.max(train_metadata['number_detect']), np.max(test_metadata['number_detect']))
print(np.mean(train_metadata['number_detect']), np.mean(test_metadata['number_detect']))

In [ ]:
print(np.min(train_metadata['len_detect']), np.min(test_metadata['len_detect']))
print(np.max(train_metadata['len_detect']), np.max(test_metadata['len_detect']))
print(np.mean(train_metadata['len_detect']), np.mean(test_metadata['len_detect']))

[Go back to top.](#index)